In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
import time
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os
current_dir = current_dir= os.getcwd()

### Funciones y para qué sirven:
- El DOM (Document Object Model o Modelo de Objetos del Documento) es una estructura en forma de árbol que representa todo el contenido HTML y XML de una página web. Básicamente, cuando cargas una página, el navegador convierte el HTML en una estructura organizada de nodos (como elementos, atributos y texto) que se puede manipular desde JavaScript o herramientas como Selenium. Gracias al DOM, puedes localizar, leer o modificar cualquier parte de la página, como hacer clic en botones, leer textos de una tabla o llenar formularios de forma automática.
- webdriver.Chrome(): Lanza una nueva ventana del navegador Chrome. Esto inicia el entorno de automatización para interactuar con páginas web.
- driver.get(URL): Carga la página web cuyo enlace se proporciona. Es como si escribieras esa URL en la barra del navegador y dieras Enter.
- WebDriverWait(driver, tiempo).until(condición): Espera a que se cumpla una condición antes de seguir. Esto es crucial en páginas dinámicas que tardan en cargar ciertos elementos. Ayuda a que el script no falle por intentar acceder a algo que aún no ha aparecido.
- EC.presence_of_element_located((By.XXX, valor)): Verifica que un elemento esté presente en el DOM, que es la estructura jerárquica del HTML de la página. Aunque el elemento no se vea en pantalla todavía, esta condición se cumple si ya está cargado internamente.
- EC.element_to_be_clickable((By.XXX, valor)): Este método espera a que un elemento esté visible y habilitado, es decir, que esté listo para recibir clics. No basta con que esté en el DOM: debe ser visible en pantalla y no estar deshabilitado, oculto o bloqueado por otro elemento.
- driver.execute_script("JavaScript"): Permite ejecutar comandos JavaScript directamente sobre la página. Es útil cuando Selenium no puede interactuar con elementos de forma estándar (por ejemplo, botones escondidos o animaciones que bloquean clics).
- ActionChains(driver): Crea una “cadena de acciones” del ratón o teclado, como mover el puntero, hacer clic o arrastrar elementos. Es útil cuando necesitas simular una interacción más realista.
- .move_to_element(element).click().perform(): Mueve el ratón hasta un elemento en la pantalla y hace clic sobre él. Se usa cuando hay menús desplegables o botones que aparecen solo al pasar el ratón encima.
- element.click(): Realiza un clic sobre el elemento. Es más directo, pero a veces falla si el elemento está cubierto o fuera de la vista.
- driver.find_element() y driver.find_elements(): Estos métodos buscan uno o varios elementos dentro del DOM. Son la base para localizar cosas como botones, campos de texto, filas de una tabla, etc.
- element.scrollIntoView(): Hace que el navegador haga scroll hasta que el elemento esté visible. Esto es necesario para interactuar con elementos que no están inicialmente en la pantalla.
- element.scrollTop = element.scrollHeight: Esta línea simula un scroll dentro de un contenedor (por ejemplo, un div con una tabla). Es útil para forzar que se carguen más datos cuando la página usa carga diferida.
- element.text: Devuelve el texto que se ve en pantalla dentro de ese elemento del DOM. Por ejemplo, puede ser el nombre de un país o un valor numérico de la tabla.

In [2]:
# Se inicializa el navegador Chrome usando Selenium WebDriver. Esto abre una ventana de Chrome controlada por el script.
driver = webdriver.Chrome()
# Se abre la página de datos del Banco Mundial en el navegador. El método get() carga la URL especificada.
driver.get("https://databank.worldbank.org/reports.aspx?source=2&series=FP.CPI.TOTL&country=")
# Se intenta cerrar un banner emergente si está presente en la página. Este bloque usa un try-except para manejar errores.
try:
    # Se utiliza WebDriverWait para esperar hasta 30 segundos a que un elemento con el valor 'Close' esté presente.
    # El XPATH "//input[@value='Close']" busca un botón de entrada con el valor 'Close'.
    WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.XPATH, "//input[@value='Close']"))
    )
    # Se ejecuta un script de JavaScript para hacer clic en el botón 'Close'. Esto evita problemas con clics directos si el elemento no es interactuable.
    driver.execute_script("document.querySelector('input[value=\"Close\"]').click();")
    # Se imprime un mensaje en la consola para confirmar que el banner se cerró correctamente.
    print("Banner cerrado")
# Si ocurre un error (por ejemplo, el banner no aparece o no se puede cerrar), se captura la excepción.
except Exception as e:
    # Se imprime un mensaje de error con detalles de la excepción.
    print("Error al cerrar el banner:", e)
# Se agrega una espera de 3 segundos para dar tiempo a que la página cargue completamente.
# Aunque no es la mejor práctica (se prefieren esperas explícitas), se usa aquí por simplicidad.
time.sleep(3)
# Se intenta interactuar con el menú desplegable de "Time" para seleccionar opciones de tiempo.
try:
    # Se espera hasta 30 segundos a que el botón con el texto 'Time' y una clase 'caret' sea clickeable.
    # El XPATH "//a[@data-text='Time']//span[@class='caret']" localiza el botón del menú desplegable.
    caret_button = WebDriverWait(driver, 30).until(
        EC.element_to_be_clickable((By.XPATH, "//a[@data-text='Time']//span[@class='caret']"))
    )
    # Se desplaza la vista del navegador hasta el botón usando JavaScript para asegurar que sea visible.
    driver.execute_script("arguments[0].scrollIntoView(true);", caret_button)
    # Se crea una instancia de ActionChains para realizar acciones avanzadas como mover el cursor.
    actions = ActionChains(driver)
    # Se mueve el cursor al botón y se hace clic en él para abrir el menú desplegable.
    actions.move_to_element(caret_button).click().perform()
    # Se imprime un mensaje para confirmar que el menú desplegable se abrió.
    print("Desplegable de 'Time' abierto")
# Si ocurre un error (por ejemplo, el botón no se encuentra), se captura la excepción.
except Exception as e:
    # Se imprime un mensaje de error con detalles de la excepción.
    print("Error al hacer clic en el caret:", e)
# Se intenta desmarcar todas las opciones de tiempo, seleccionar el año 2018 y aplicar los cambios.
try:
    # Se espera hasta 30 segundos a que el botón "Unselect All Time" sea clickeable.
    # El XPATH "//a[@data-text='UnSelect All Time']" localiza el botón para desmarcar todas las opciones.
    unselect_all_button = WebDriverWait(driver, 30).until(
        EC.element_to_be_clickable((By.XPATH, "//a[@data-text='UnSelect All Time']"))
    )
    # Se hace clic en el botón para desmarcar todas las opciones de tiempo.
    unselect_all_button.click()
    # Se imprime un mensaje para confirmar que se desmarcaron todas las opciones.
    print("Botón 'Unselect all' clickeado")
    # Se espera hasta 30 segundos a que el checkbox para el año 2018 sea clickeable.
    # El ID "chk[WDI_Time].[Year].&[YR2018]" identifica el checkbox correspondiente al año 2018.
    checkbox = WebDriverWait(driver, 30).until(
        EC.element_to_be_clickable((By.ID, "chk[WDI_Time].[Year].&[YR2018]"))
    )
    # Se hace clic en el checkbox para seleccionar el año 2018.
    checkbox.click()
    # Se imprime un mensaje para confirmar que se seleccionó el checkbox.
    print("Checkbox clickeado")
    # Se espera hasta 30 segundos a que el botón "Apply Changes" sea clickeable.
    # El XPATH busca un enlace con la clase específica y el atributo 'data-trackaction'.
    apply_changes_button = WebDriverWait(driver, 30).until(
        EC.element_to_be_clickable((By.XPATH, "//a[@class='actionBtn pull-right applyChangesDim' and @data-trackaction='apply changes']"))
    )
    # Se hace clic en el botón para aplicar los cambios seleccionados.
    apply_changes_button.click()
    # Se imprime un mensaje para confirmar que se aplicaron los cambios.
    print("Botón 'Apply Changes' clickeado")
# Si ocurre un error (por ejemplo, un elemento no se encuentra), se captura la excepción.
except Exception as e:
    # Se imprime un mensaje de error con detalles de la excepción.
    print("Error al interactuar con los botones:", e)
# Se agrega una espera de 5 segundos para asegurar que los cambios se apliquen y la página se actualice.
time.sleep(5)
# Se localiza el contenedor de la tabla de datos usando su clase CSS 'dxgvCSD'.
dxgvCSD_div = driver.find_element(By.CLASS_NAME, "dxgvCSD")
# Se desplaza la vista del navegador hasta el contenedor para que sea visible.
driver.execute_script("arguments[0].scrollIntoView(true);", dxgvCSD_div)
# Se realiza un bucle para desplazar el contenedor hacia abajo 5 veces, cargando más datos si están disponibles.
for _ in range(5):  # El número de iteraciones (5) puede ajustarse según la cantidad de datos a cargar.
    # Se usa JavaScript para desplazar el contenedor hasta el final de su altura.
    driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", dxgvCSD_div)
    # Se espera 2 segundos después de cada desplazamiento para permitir que los datos se carguen.
    time.sleep(2)  # Espera a que los datos se carguen
# Se intenta extraer los datos de la tabla después de que se haya cargado.
try:
    # Se espera hasta 30 segundos a que la tabla con el ID 'grdTableView_DXMainTable' sea visible.
    WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.XPATH, "//table[@id='grdTableView_DXMainTable']"))
    )
    # Se obtienen todas las filas de la tabla usando el XPATH que selecciona las etiquetas <tr> dentro de la tabla.
    filas = driver.find_elements(By.XPATH, "//table[@id='grdTableView_DXMainTable']//tr")
    # Se crea una lista vacía para almacenar los datos extraídos.
    datos = []
    # Se itera sobre cada fila de la tabla.
    for fila in filas:
        # Se obtienen todas las celdas (<td>) de la fila actual.
        celdas = fila.find_elements(By.TAG_NAME, "td")
        # Se verifica que la fila tenga al menos 2 celdas (país e índice).
        if len(celdas) > 1:
            # Se extrae el texto de la primera celda (país).
            pais = celdas[0].text
            # Se extrae el texto de la segunda celda (índice de confianza del consumidor).
            indice = celdas[1].text
            # Se agrega una tupla con el país y el índice a la lista de datos.
            datos.append((pais, indice))
    # Se crea una nueva lista para almacenar los datos filtrados.
    datos_modificados = []
    # Se itera sobre los datos extraídos para filtrar valores vacíos o inválidos.
    for pais, numero in datos:
        # Se verifica que el país no esté vacío y que el índice no sea '..' (valor inválido).
        if pais != '' and numero != '..':  # Filtrar los países sin datos
            # Se agrega la tupla filtrada a la lista de datos modificados.
            datos_modificados.append((pais, numero))
    # Se imprimen los encabezados de la tabla para mostrar los datos de forma clara.
    print("País", "Índice de confianza del consumidor (2018)")
    # Se itera sobre los datos filtrados para imprimir cada par país-índice.
    for pais, indice in datos_modificados:
        # Se imprime el país y el índice en un formato legible.
        print(f"País: {pais}, Índice de confianza del consumidor (2018): {indice}")
# Si ocurre un error (por ejemplo, la tabla no se encuentra), se captura la excepción.
except Exception as e:
    # Se imprime un mensaje de error con detalles de la excepción.
    print("Error al extraer los datos de la tabla:", e)
# Se cierra el navegador y se liberan los recursos del sistema.
# Esto asegura que no queden procesos abiertos después de la ejecución.
driver.quit()

Banner cerrado
Desplegable de 'Time' abierto
Botón 'Unselect all' clickeado
Checkbox clickeado
Botón 'Apply Changes' clickeado
País Índice de confianza del consumidor (2018)
País: Afghanistan, Índice de confianza del consumidor (2018): 146.5
País: Albania, Índice de confianza del consumidor (2018): 117.4
País: Algeria, Índice de confianza del consumidor (2018): 148.5
País: Angola, Índice de confianza del consumidor (2018): 324.2
País: Antigua and Barbuda, Índice de confianza del consumidor (2018): 113.8
País: Armenia, Índice de confianza del consumidor (2018): 127.3
País: Aruba, Índice de confianza del consumidor (2018): 105.1
País: Australia, Índice de confianza del consumidor (2018): 117.9
País: Austria, Índice de confianza del consumidor (2018): 116.3
País: Azerbaijan, Índice de confianza del consumidor (2018): 152.9
País: Bahamas, The, Índice de confianza del consumidor (2018): 113.4
País: Bahrain, Índice de confianza del consumidor (2018): 117.6
País: Bangladesh, Índice de confian

In [3]:
ICC = pd.DataFrame(datos_modificados)
ICC.columns = ["País", "Índice de confianza del consumidor (2018)"]

In [4]:
## Guardar en CSV
# Path
indice_confianza_consumidor_path = os.path.join(current_dir, '../../data/raw/macrodata')
# Crear un directorio si no existe
os.makedirs(indice_confianza_consumidor_path, exist_ok=True)
# Archivo CSV
indice_confianza_consumidor = os.path.join(indice_confianza_consumidor_path, "indice_confianza_consumidor_2018.csv")
# Guardar dataframe en CSV
ICC.to_csv(indice_confianza_consumidor, index=False)